<a href="https://colab.research.google.com/github/Predic-learning/Buiding-Energy/blob/main/Floor_Poly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error, classification_report
data = pd.read_csv('/content/drive/MyDrive/weather/Fl-5.csv')
#data['Tools'].replace({'AIR':0, 'PC':1 , 'lamp':2}, inplace=True)
data.head()

,Class,Tools,Room,Number,BTU,Power,amount,PowerHr,time,SumPower
0,5,AIR,501,1872,38000,3.3,1,3.3,39,128.7
1,5,AIR,502,1873,38000,3.3,1,3.3,39,128.7
2,5,AIR,503,1874,38000,3.3,1,3.3,39,128.7
3,5,AIR,502,1869,38000,3.3,1,3.3,39,128.7
4,5,AIR,503,1870,38000,3.3,1,3.3,39,128.7


In [2]:
num_points = 368
max_error_percent = 20
randomP = [random.uniform(-3.3, 3.3) for _ in range(num_points)]
errorX = [valueX * (max_error_percent / 100) * random.choice([-1, 1]) for valueX in randomP]
dfX = pd.DataFrame({'random_dataX': errorX})
dfX['absolute_random_dataX'] = dfX['random_dataX'].abs()
random_Power = data['PowerHr'] - dfX['absolute_random_dataX']
data = data.assign(random_Power = random_Power)

randomS = [random.uniform(-128.7, 128.7) for _ in range(num_points)]
error = [value * (max_error_percent / 100) * random.choice([-1, 1]) for value in randomS]
df = pd.DataFrame({'random_data': error})
df['absolute_random_data'] = df['random_data'].abs()
random_SumPower = data['SumPower'] - df['absolute_random_data']
data = data.assign(random_SumPower = random_SumPower)
data

,Class,Tools,Room,Number,BTU,Power,amount,PowerHr,time,SumPower,random_Power,random_SumPower
0,5,AIR,501,1872,38000,3.30,1,3.300,39,128.70,2.733258,119.480717
1,5,AIR,502,1873,38000,3.30,1,3.300,39,128.70,3.072379,110.668700
2,5,AIR,503,1874,38000,3.30,1,3.300,39,128.70,2.911818,117.254456
3,5,AIR,502,1869,38000,3.30,1,3.300,39,128.70,3.214270,103.035416
4,5,AIR,503,1870,38000,3.30,1,3.300,39,128.70,3.044881,107.458946
5,5,AIR,504,1871,38000,3.30,1,3.300,39,128.70,3.243040,105.706284
6,5,AIR,503,1866,38000,3.30,1,3.300,36,118.80,2.924211,101.961882
7,5,AIR,504,1867,38000,3.30,1,3.300,36,118.80,3.214767,116.002511
8,5,AIR,505,1868,38000,3.30,1,3.300,36,118.80,3.173798,100.120891
9,5,AIR,504,1863,38000,3.30,1,3.300,30,99.00,2.971165,81.004004


In [3]:
#X = data[['PowerHr','time']].values
#y = data['SumPower']
X = data[['time', 'random_Power']].values
y = data[['random_SumPower']].values.reshape(-1, 1)
poly = PolynomialFeatures(degree=2)
X_poly = poly.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
model = LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

# Evaluate the model using regression metrics
mse = mean_squared_error(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse:.2f}")
print(f"Mean Absolute Error (MAE): {mae:.2f}")
print(f"Root Mean Squared Error (RMSE): {rmse:.2f}")
print(f"Accuracy: {r2*100:.2f} %")

Mean Squared Error (MSE): 91.90
Mean Absolute Error (MAE): 8.25
Root Mean Squared Error (RMSE): 9.59
Accuracy: 92.69 %


In [5]:
T= 1
Data = [[data.loc[T, 'time'], data.loc[T, 'random_Power']]]
y_predict = model.predict(Data)
y_predict

array([[119.42472329]])